In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import mysql.connector
import pandas as pd

import json

with open('config.json') as f:
    config = json.load(f)


cnx = mysql.connector.connect(user=config['user'],
                              password=config['password'],
                              host=config['host'],
                              database=config['database'])

cursor = cnx.cursor()

# Example query
query = ("SELECT * FROM PH_Temp_Diet")

cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

# Create a pandas DataFrame
data = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])


cursor.close()
cnx.close()

In [ ]:
df3 =data
df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')
pivoted_data = pivoted_data.drop('person_id', axis=1)
pivoted_data = pivoted_data.reset_index()
# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data = joined_data.drop_duplicates()
joined_data

In [ ]:
from sklearn.model_selection import train_test_split

X= joined_data.iloc[:,0:1]
X

In [ ]:
y = joined_data.iloc[:,5:]
y

In [ ]:
y = data.groupby(['person_id', 'date'])[['food_name', 'amount']].agg(list)
# Reset the index
y = y.reset_index()

# Custom function to combine food_name and amount into one list
def combine_food_and_amount(food_names, amounts):
    combined = []
    for food, amount in zip(food_names, amounts):
        if amount != 0:
            amount = 1
        combined.append((food, amount))
    return combined

# Apply the custom function to combine food_name and amount columns
y['food_and_amount'] = y.apply(lambda row: combine_food_and_amount(row['food_name'], row['amount']), axis=1)

# Drop the food_name and amount columns
y = y.drop(['food_name', 'amount','person_id','date'], axis=1)

# Print the modified DataFrame
print(y.head(5))

In [ ]:
df = pd.DataFrame(y)
df2 = pd.DataFrame(X)
# convert column 'A' to a tuple
food_and_amount = tuple(df['food_and_amount'].tolist())
ph = tuple(df2['PH'].tolist())
print(ph)

In [ ]:
food_and_amount

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

data = pd.DataFrame({
    'pH_level': ph,
    'food_and_activities': food_and_amount
})

def categorize_pH(pH):
    if pH < 7.0:
        return "acidic"
    elif pH > 7.0:
        return "alkaline"
    else:
        return "neutral"

data['pH_category'] = data['pH_level'].apply(categorize_pH)

def binary_representation(row):
    binary_list = [row['pH_category']]
    for food, amount in row['food_and_activities']:
        if amount > 0:
            binary_list.append(food)
    return binary_list

data['binary'] = data.apply(binary_representation, axis=1)

transactions = data['binary'].tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
binary_df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(binary_df, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

# Filter the rules based on the antecedent (left-hand side) containing the pH category "acidic"
acidic_rules = rules[rules['antecedents'].apply(lambda x: 'acidic' in x)]
alkaline_rules = rules[rules['antecedents'].apply(lambda x: 'alkaline' in x)]

print(alkaline_rules)


In [ ]:
def recommend_food_and_activities(pH_level, rules):
    pH_category = categorize_pH(pH_level)

    # Filter the rules based on the antecedent (left-hand side) containing the pH category
    recommendations = rules[rules['antecedents'].apply(lambda x: pH_category in x)]

    # Sort the rules by confidence and select the top 5 rules
    top_recommendations = recommendations.sort_values(by='confidence', ascending=False).head(5)

    # Extract the consequents (right-hand side) of the rules
    recommended_items = set()
    for _, row in top_recommendations.iterrows():
        recommended_items.update(row['consequents'])

    # Remove the pH category from the recommended items
    recommended_items.discard(pH_category)

    return list(recommended_items)


In [ ]:
# Example usage
urine_pH = 9 # Replace this with the actual pH level data from the sensor
recommended_items = recommend_food_and_activities(urine_pH, alkaline_rules)
print("Recommended food and activities:", recommended_items)

In [ ]:
import joblib

# Save the association rules DataFrame to a file
joblib.dump(alkaline_rules, 'alkaline_rules.joblib')

In [27]:
import joblib

# Load the saved association rules DataFrame from a file
loaded_alkaline_rules = joblib.load('alkaline_rules.joblib')

# Use the loaded rules in the recommend_food_and_activities function
urine_pH = 9  # Replace this with the actual pH level data from the sensor
recommended_items = recommend_food_and_activities(urine_pH, loaded_alkaline_rules)
print("Recommended food and activities:", recommended_items)




Recommended food and activities: ['Liquid', 'Fruit tea']
